# Navigation

---

Congratulations for completing the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893)!  In this notebook, you will learn how to control an agent in a more challenging environment, where it can learn directly from raw pixels!  **Note that this exercise is optional!**

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/VisualBanana.app"`
- **Windows** (x86): `"path/to/VisualBanana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/VisualBanana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/VisualBanana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/VisualBanana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/VisualBanana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `VisualBanana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="VisualBanana.app")
```

In [ ]:
env =  UnityEnvironment(file_name="./VisualBanana_Windows_x86_64/Banana.exe")

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The environment state is an array of raw pixels with shape `(1, 84, 84, 3)`.  *Note that this code differs from the notebook for the project, where we are grabbing **`visual_observations`** (the raw pixels) instead of **`vector_observations`**.* A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.visual_observations[0]
print('States look like:')
plt.imshow(np.squeeze(state))
plt.show()
state_size = state.shape
print('States have shape:', state.shape)

In [ ]:
state_size

In [ ]:
env_info = env.reset()

In [ ]:
state=env.step(1)

In [ ]:
env_info.visual_observations

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [ ]:
env_info = env.reset(train_mode=False)[brain_name] # reset the environment
state = env_info.visual_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.visual_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [ ]:
from unityagents import UnityEnvironment
import numpy as np
import random
import matplotlib.pyplot as plt
from collections import deque
from IPython.display import clear_output
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
ewma = lambda x: pd.Series(x).ewm(span=100).mean()

from agent import QAgent
from models import CNNArchitecture

%matplotlib inline

In [ ]:
env = UnityEnvironment(file_name="./VisualBanana_Windows_x86_64/Banana.exe")
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
state_size = brain.vector_observation_space_size
n_actions = brain.vector_action_space_size
env_info = env.reset(train_mode=True)[brain_name]
state = env_info.visual_observations[0]
state_size = state.shape[1:]

In [ ]:
#%%timeit
n_episodes = 1500
eps_decay=0.99995
eps_final=0.01
exploration_steps = 10000
tau = 1e-3
learning_rate = 1e-3
exp_replay_size = int(1e5)
update_every = 4
agent = QAgent(n_actions=n_actions, state_size=state_size, architecture=CNNArchitecture,
               epsilon_exp_decay=eps_decay, epsilon_final=eps_final, batch_size=64,
               initial_exploration_steps=exploration_steps, tau = tau,
               update_every=update_every,exp_replay_size=exp_replay_size)
scores = []
eps = []
for i in range(n_episodes):
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.visual_observations[0]  
    done=False
    episodic_score=0
    while not done:
        action = agent.choose_action(state)

        env_info = env.step(int(action))[brain_name]
        reward = env_info.rewards[0]
        next_state = env_info.visual_observations[0]
        done = env_info.local_done[0] 

        agent.step(state=state, action=action, reward=reward, next_state=next_state)
        state = next_state
        episodic_score += reward
    eps.append(agent.epsilon)
    scores.append(episodic_score)
    if i % 100 == 0:
        torch.save(agent.neural_net.net.state_dict(), 'checkpoint.pth')
            
    clear_output(True)
    plt.figure(figsize=(12, 5))
    plt.subplot(121)
    plt.plot(ewma(scores))
    plt.grid()
    plt.xlabel("Episodes")
    plt.ylabel("Cumulative reward")
    plt.title("Agent performance evolution")
    plt.subplot(122)
    plt.plot(eps)
    plt.grid()
    plt.xlabel("Episodes")
    plt.ylabel("Epsilon")
    plt.title("Epsilon evolution over episodes")
    plt.show()


In [ ]:
state.swapaxes(1,3).shape

In [ ]:
%debug

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.visual_observations[0]
print('States look like:')
plt.imshow(np.squeeze(state))
plt.show()
state_size = state.shape
print('States have shape:', state.shape)